In [1]:
import sys
import numpy as np
from matplotlib import pyplot as plt
import h5py

%matplotlib notebook

In [2]:
import qcodes as qc

# Setup notes

For this example to run, you need to have set up a labpython config file. There is a template in labpythonQDelft/templates.

In [15]:
from importlib import reload
from qctools import instruments as instools
reload(instools)

import labpythonconfig as cfg
reload(cfg)

from experiment import measurement
reload(measurement)
from experiment.measurement import Parameter, BaseMeasurement, PysweepMeasurement

In [19]:
from pysweep.data_storage import np_storage
reload(np_storage)
from pysweep.data_storage.np_storage import NpStorage, DataSet

from pysweep import sweep_object
reload(sweep_object)
from pysweep.sweep_object import sweep, ChainSweep

In [17]:
station = qc.Station()

In [18]:
class NameSpace:
    pass

namespace = NameSpace()

In [7]:
class DummyMeasurement(BaseMeasurement):
    def __init__(self, station, namespace):
        super().__init__(station, namespace)
        
        self.add_parameter('start', Parameter)
        self.add_parameter('stop', Parameter)
        self.add_parameter('npts', Parameter)
    
    def measure(self):        
        # set up loop
        xpoints = np.linspace(self.start(), self.stop(), self.npts())
        ypoints = np.linspace(self.start(), self.stop(), self.npts())
                
        # now populate in a primitive loop
        for x in xpoints:
            for y in ypoints:
                
                # acquire some data
                z = np.cos(x) * np.cos(y) * np.linspace(0,1,3)

                # add to dataset
                self.data.add(
                    {'x' : {'value' : x, 'independent_parameter' : True, 'unit' : 'V'},
                     'y' : {'value' : y, 'independent_parameter' : True, 'unit' : 'V'},
                     'z' : {'value' : z },
                    }
                )
                    

m = DummyMeasurement(station, namespace)
m.start(0)
m.stop(10)
m.npts(3)
m.run()

In [21]:
voltage1 = 0
voltage2 = 0

def getv1():
    global voltage1
    return voltage1

def setv1(v):
    global voltage1
    voltage1 = v

def getv2():
    global voltage2
    return voltage2

def setv2(v):
    global voltage2
    voltage2 = v

v1 = qc.Parameter('voltage1', get_cmd=getv1, set_cmd=setv1)
v2 = qc.Parameter('voltage2', get_cmd=getv2, set_cmd=setv2)

station = qc.Station(v1, v2)

In [22]:
class SimplePysweepMeasurement(PysweepMeasurement):

    def measure_datapoint(self, station, namespace):
        return {'data' : {'value' : self.station.voltage1() * self.station.voltage2()**2 }}
        
    def sweep(self):
        return sweep(self.station.voltage1, np.linspace(0,1,3)) (
            sweep(self.station.voltage2, np.linspace(0,2,5)) (
                self.measure_datapoint,
            )
        )
    
m = SimplePysweepMeasurement(station, namespace)
m.run()

In [23]:
fn = m.datafilepath
print(fn)

c:\data\2017-12\2017-12-22\2017-12-22_SimplePysweepMeasurement-#0001.hdf5
